<a href="https://colab.research.google.com/github/Himanshu-Tagde/Assignment/blob/main/Amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!apt-get update
!apt-get install -y wget curl unzip
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt --fix-broken install -y
!pip install -q selenium
!pip install -q chromedriver-autoinstaller

# import the necessary libraries
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading

,Product Name,Price,Manufacturer,Dimensions,Weight,Warranty,Colour,Features,Brand,Model,Capacity,Installation Type,Part Number,Special Features,Control Console,Access Location,Included Components,Batteries Required,Country of Origin
0,Acer 10.0 Kg 5 Star Semi-Automatic Top Loading...,"11,999.",‎Dixon Technologies (India) Limited,91.5 x 101.5 x 56 Centimeters,28 kg,2 years comprehensive and 5 years on motor.,‎Black-Grey,Semi Automatic Top Load washing machine: User-...,‎acer,‎AR10SAMBGL124D,‎10 Kilograms,‎Freestanding,‎AR10SAMBGL124D,"‎Protective Rat Mesh, Auto Restart, High Effic...",‎Push Button,‎Top Load,"‎DRAIN CLAMP, DRAIN PIPE, SAFETY COVER, INLET ...",‎No,‎India


In [ ]:

def get_amazon_product_details(url):
    # chromedriver
    chromedriver_autoinstaller.install()

    # Set up Chrome options
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-blink-features=AutomationControlled")

    # Launch Chrome with the options
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(3)

    # Parse the page with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    # Extract product details
    product_details = {
        "Product Name": soup.find("span", {"id": "productTitle"}).text.strip() if soup.find("span", {"id": "productTitle"}) else None,
        "Price": soup.find("span", {"class": "a-price-whole"}).text.strip() if soup.find("span", {"class": "a-price-whole"}) else None,
        "Manufacturer": None,
        "Dimensions": None,
        "Weight": None,
        "Warranty": None,
        "Color": None,
        "Features": [],
        "Other Information": {},
    }

    # Possible table IDs for product details
    table_ids = ["productDetails_detailBullets_sections1", "productDetails_techSpec_section_1"]

    for table_id in table_ids:
        details_table = soup.find("table", {"id": table_id})
        if details_table:
            for row in details_table.find_all("tr"):
                key = row.find("th").text.strip()
                value = row.find("td").text.strip()
                if "Manufacturer" in key:
                    product_details["Manufacturer"] = value
                elif "Dimensions" in key:
                    product_details["Dimensions"] = value
                elif "Item Weight" in key:
                    product_details["Weight"] = value
                elif "Warranty" in key:
                    product_details["Warranty"] = value
                elif "Color" in key:
                    product_details["Color"] = value
                else:
                    product_details["Other Information"][key] = value


    feature_list = soup.find("ul", {"class": "a-unordered-list a-vertical a-spacing-mini"})
    if feature_list:
        product_details["Features"] = [li.text.strip() for li in feature_list.find_all("span", {"class": "a-list-item"})]

    return product_details


amazon_url = "https://www.amazon.in/Samsung-Semi-Automatic-Washing-WT80C4200GG-TL/dp/B0CVQ8DX8H/ref=sr_1_2_sspa?dib=eyJ2IjoiMSJ9.6U1qFgk58BwtShX68xP1cT7QAcmecplx-edjZvBW-snbT5xCXVmDUpaXkXBglUqYZ9JVtyCqIYguiWSDDLCPD8A_B5vwYOIDNRx_GYOSnxPbT20pvViTkFMFK90X-XlMKqo7514yvuecMmqW5JIqrjFktIJ9LUJZi9OJrzUO4s0wo5yLiQjdBtS_ZFM1KQDOylKp-0raYFHASZmg5Rmk-kxc_VzNCJITkT57R4hPE5I.rYPhUzRErOeLGHey2ACPOIMPTpDTFRDZRpnQOzi7QEM&dib_tag=se&keywords=washing%2Bmachine%2B8kg&qid=1745043278&sr=8-2-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&th=1"

# Fetch product information
product_info = get_amazon_product_details(amazon_url)

# Extract warranty info
warranty_text = None
for feature in product_info.get('Features', []):
    if feature.startswith("Warranty:"):
        warranty_text = feature.replace("Warranty: ", "")

# Create a DataFrame to display the product info
product_df = pd.DataFrame([{
    "Product Name": product_info["Product Name"],
    "Price": product_info["Price"],
    "Manufacturer": product_info["Manufacturer"],
    "Dimensions": product_info["Dimensions"],
    "Weight": product_info["Weight"],
    "Warranty": warranty_text,
    "Colour": product_info["Other Information"].get("Colour", None),
    "Features": "; ".join(product_info["Features"]),
    "Brand": product_info["Other Information"].get("Brand", None),
    "Model": product_info["Other Information"].get("Model", None),
    "Capacity": product_info["Other Information"].get("Capacity", None),
    "Installation Type": product_info["Other Information"].get("Installation Type", None),
    "Part Number": product_info["Other Information"].get("Part Number", None),
    "Special Features": product_info["Other Information"].get("Special Features", None),
    "Control Console": product_info["Other Information"].get("Control Console", None),
    "Access Location": product_info["Other Information"].get("Access Location", None),
    "Included Components": product_info["Other Information"].get("Included Components", None),
    "Batteries Required": product_info["Other Information"].get("Batteries Required", None),
    "Country of Origin": product_info["Other Information"].get("Country of Origin", None)
}])

# Display the DataFrame
product_df


,Product Name,Price,Manufacturer,Dimensions,Weight,Warranty,Colour,Features,Brand,Model,Capacity,Installation Type,Part Number,Special Features,Control Console,Access Location,Included Components,Batteries Required,Country of Origin
0,"Samsung 8 kg, 5 star, Semi-Automatic Top Load ...","12,990.",‎Samsung India Pvt ltd,48.5 x 83 x 103 Centimeters,24 kg 500 g,2 years on product and 2 years on motor,None,Semi-Automatic top load washing maching: Econo...,None,‎WT80C4200GG/TL,None,None,‎WT80C4200GG/TL,"‎Inverter, High Efficiency",None,None,"‎‎1 Washing Machine, Warranty Card and User Ma...",‎No,‎India


In [ ]:
# Save the DataFrame to an Excel file
file_name = "/content/amazon_product_details.xlsx"
product_df.to_excel(file_name, index=False, engine="openpyxl")
from google.colab import files
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>